In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [2]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Custom Dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        x = self.data[index:index+self.seq_length]
        y = self.data[index+self.seq_length]
        return x, y

In [3]:
# Load and preprocess the data
df = pd.read_csv('spi_results.csv')
# features = ['tmax_m', 'tmax_max', 'tmax_min', 'tmin_m', 'tmin_min', 'tmin_max', 'ntmin_0', 'rrr24', 'sshn', 'tm_m', 't_03_m', 't_09_m', 't_15_m']
features = ['tmax_m', 'tmin_m', 'rrr24', 'SPI']

target = 'SPI'

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[features])
scaled_target = scaler.fit_transform(df[[target]])

data = torch.tensor(scaled_features, dtype=torch.float32)
target = torch.tensor(scaled_target, dtype=torch.float32)

# Hyperparameters
input_size = len(features)
hidden_size = 64
num_layers = 2
output_size = 1
seq_length = 10
batch_size = 32
num_epochs = 100
learning_rate = 0.001

# Create DataLoader
dataset = TimeSeriesDataset(data, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Training loop
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'lstm_model.pth')
# Load the saved model for inference
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

# Prepare test data for prediction
test_data = torch.tensor(scaled_features[-seq_length:], dtype=torch.float32).unsqueeze(0)

# Make predictions
with torch.no_grad():
    predictions = model(test_data)

# Inverse transform predictions to original scale
predictions_rescaled = scaler.inverse_transform(predictions.numpy())

# Print the predictions
print("Predicted SPI:", predictions_rescaled)


c:\Users\varas\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([32, 4])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\varas\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([27, 4])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.0881
Epoch [20/100], Loss: 0.0637
Epoch [30/100], Loss: 0.0804
Epoch [40/100], Loss: 0.0889
Epoch [50/100], Loss: 0.0752
Epoch [60/100], Loss: 0.0903
Epoch [70/100], Loss: 0.0963
Epoch [80/100], Loss: 0.0620
Epoch [90/100], Loss: 0.0893
Epoch [100/100], Loss: 0.0892
Predicted SPI: [[-0.0811561]]


C:\Users\varas\AppData\Local\Temp\ipykernel_21644\1625082578.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('lstm_model.pth'))
